In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [3]:
from loadFile import getFileList, getDirList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "enron/maildir/allen-p/sent"
file_list = getFileList(directory_path)

# テキストファイルをデータフレームに格納する
df = fileToDataFrame(file_list)

In [4]:
df.head()

,Path,Message-ID,Date,From,To,Cc,Bcc,X-From,X-To,X-cc,X-bcc,Subject,Message
0,enron/maildir/allen-p/sent/36.,<3475906.1075855680543.JavaMail.evans@thyme>,"Tue, 31 Oct 2000 04:44:00 -0800 (PST)",phillip.allen@enron.com,stephen.stock@enron.com,None,None,Phillip K Allen,Stephen Stock,,,Re: Astral downtime request,"Steve,\n\nThank you for the update. The need is still great for this disk space. \n\nPhillip"
1,enron/maildir/allen-p/sent/313.,<28759434.1075855713622.JavaMail.evans@thyme>,"Tue, 1 May 2001 07:14:00 -0700 (PDT)",phillip.allen@enron.com,ina.rangel@enron.com,None,None,Phillip K Allen,Ina Rangel,,,4-URGENT - OWA Please print this now.,"---------------------- Forwarded by Phillip K Allen/HOU/ECT on 05/01/2001 \n02:14 PM ---------------------------\n\n\nOutlook Migration Team@ENRON\n04/27/2001 01:01 PM\nTo: Allison Horton/NA/Enron@ENRON, Amir Baig/NA/Enron@ENRON, Brandon \nBangerter/NA/Enron@Enron, Brian Ellis/Corp/Enron@Enron, Charles \nPhilpott/HR/Corp/Enron@ENRON, Chris P Wood/NA/Enron@Enron, Chris \nTull/HOU/ECT@ECT, Dale Smith/Corp/Enron@ENRON, Dave June/NA/Enron@ENRON, \nDonald Sutton/NA/Enron@Enron, Felicia Buenrostro/HR/Corp/Enron@ENRON, Johnna \nMorrison/Corp/Enron@ENRON, Joe Dorn/Corp/Enron@ENRON, Kathryn \nSchultea/HR/Corp/Enron@ENRON, Leon McDowell/NA/Enron@ENRON, Leticia \nBarrios/Corp/Enron@ENRON, Milton Brown/HR/Corp/Enron@ENRON, Raj \nPerubhatla/Corp/Enron@Enron, Shekar Komatireddy/NA/Enron@Enron, Andrea \nYowman/Corp/Enron@ENRON, Angie O'Brian/HR/Corp/Enron@ENRON, Bonne \nCastellano/HR/Corp/Enron@ENRON, Gwynn Gorsuch/NA/Enron@ENRON, Jo Ann \nMatson/Corp/Enron@ENRON, LaQuitta Washington/HR/Corp/Enro..."
2,enron/maildir/allen-p/sent/264.,<29864817.1075855685505.JavaMail.evans@thyme>,"Mon, 21 Feb 2000 23:03:00 -0800 (PST)",phillip.allen@enron.com,jsmith@austintx.com,None,None,Phillip K Allen,"""Jeff Smith"" <jsmith@austintx.com>",,,Re: 91acres,"Jeff,\n\n Let's just close on March 1.\n\nPhillip"
3,enron/maildir/allen-p/sent/166.,<13269622.1075855683379.JavaMail.evans@thyme>,"Mon, 17 Jul 2000 03:46:00 -0700 (PDT)",phillip.allen@enron.com,paul.lucci@enron.com,None,None,Phillip K Allen,Paul T Lucci,,,Comments on Order 637 Compliance Filings,"fyi CIG\n\n---------------------- Forwarded by Phillip K Allen/HOU/ECT on 07/17/2000 \n10:45 AM ---------------------------\n \n\tEnron North America Corp.\n\t\n\tFrom: Rebecca W Cantrell 07/14/2000 02:31 PM\n\t\n\nTo: Julie A Gomez/HOU/ECT@ECT, Stephanie Miller/Corp/Enron@ENRON, Chris \nMeyer/HOU/ECT@ECT, Judy Townsend/HOU/ECT@ECT, Theresa Branney/HOU/ECT@ECT, \nPaul T Lucci/DEN/ECT@Enron, Jane M Tholt/HOU/ECT@ECT, Steven P \nSouth/HOU/ECT@ECT, Frank Ermis/HOU/ECT@ECT, Susan W Pereira/HOU/ECT@ECT, \nGeorge Smith/HOU/ECT@ECT, Randall L Gay/HOU/ECT@ECT, Jim Homco/HOU/ECT@ECT, \nColleen Sullivan/HOU/ECT@ECT, Hunter S Shively/HOU/ECT@ECT, Ray \nHamman/HOU/EES@EES, Robert Superty/HOU/ECT@ECT, Edward Terry/HOU/ECT@ECT, \nScott Neal/HOU/ECT@ECT, Tori Kuykendall/HOU/ECT@ECT, Brenda H \nFletcher/HOU/ECT@ECT, Jeff Coates/HOU/EES@EES, John Hodge/Corp/Enron@ENRON, \nJanet Edwards/Corp/Enron@ENRON, Ruth Concannon/HOU/ECT@ECT, Sylvia A \nCampos/HOU/ECT@ECT, Paul Ta..."
4,enron/maildir/allen-p/sent/412.,<4050541.1075855715779.JavaMail.evans@thyme>,"Thu, 8 Mar 2001 06:46:00 -0800 (PST)",ina.rangel@enron.com,information.management@enron.com,None,None,Ina Rangel,Information Risk Management,,,Mike Grigsby,"Please approve Mike Grigsby for Bloomberg. \n\nThank You,\nPhillip Allen"


In [5]:
#ドキュメント（メッセージ）をセンテンスに分解する
from preProcess import toSentences

sent_dict = {'Message-ID':[],
            'Sentence-ID':[],
            'Sentence':[]}

start = time.time()

for i,message in enumerate(df['Message']):
    sentences = toSentences(message)
    sent_dict['Message-ID'].extend([df['Message-ID'][i]] * len(sentences))
    sent_dict['Sentence-ID'].extend(range(len(sentences)))
    sent_dict['Sentence'].extend(sentences)

sentence_df = pd.DataFrame.from_dict(sent_dict,orient='columns')

end = time.time()
print(end-start,'s')

0.36233067512512207 s


In [7]:
sentence_df['Sentence'][3]

"---------------------- Forwarded by Phillip K Allen/HOU/ECT on 05/01/2001 02:14 PM --------------------------- Outlook Migration Team@ENRON 04/27/2001 01:01 PM To: Allison Horton/NA/Enron@ENRON, Amir Baig/NA/Enron@ENRON, Brandon Bangerter/NA/Enron@Enron, Brian Ellis/Corp/Enron@Enron, Charles Philpott/HR/Corp/Enron@ENRON, Chris P Wood/NA/Enron@Enron, Chris Tull/HOU/ECT@ECT, Dale Smith/Corp/Enron@ENRON, Dave June/NA/Enron@ENRON, Donald Sutton/NA/Enron@Enron, Felicia Buenrostro/HR/Corp/Enron@ENRON, Johnna Morrison/Corp/Enron@ENRON, Joe Dorn/Corp/Enron@ENRON, Kathryn Schultea/HR/Corp/Enron@ENRON, Leon McDowell/NA/Enron@ENRON, Leticia Barrios/Corp/Enron@ENRON, Milton Brown/HR/Corp/Enron@ENRON, Raj Perubhatla/Corp/Enron@Enron, Shekar Komatireddy/NA/Enron@Enron, Andrea Yowman/Corp/Enron@ENRON, Angie O'Brian/HR/Corp/Enron@ENRON, Bonne Castellano/HR/Corp/Enron@ENRON, Gwynn Gorsuch/NA/Enron@ENRON, Jo Ann Matson/Corp/Enron@ENRON, LaQuitta Washington/HR/Corp/Enron@ENRON, Rick Johnson/HR/Corp/Enro

In [9]:
from extraction import tripleExtraction, entityExtraction, NERtripleExtraction

triple_dict = {'Message-ID':[],
             'Sentence-ID':[],
             'Triple':[]}

start = time.time()

for i, sentence in enumerate(sentence_df['Sentence']):
    triples = tripleExtraction(sentence)
    entities = entityExtraction(sentence)
    nertriples = NERtripleExtraction(triples, entities)
    if nertriples is None:
        pass
    else:
        triple_dict['Message-ID'].extend([sentence_df['Message-ID'][i]] * len(nertriples))
        triple_dict['Sentence-ID'].extend([sentence_df['Sentence-ID'][i]] * len(nertriples))
        triple_dict['Triple'].extend(nertriples)

triple_df = pd.DataFrame.from_dict(triple_dict,orient='columns')

end = time.time()
print(end-start,'s')

822.5015578269958 s


In [11]:
triple_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Message-ID   217 non-null    object
 1   Sentence-ID  217 non-null    int64 
 2   Triple       217 non-null    object
dtypes: int64(1), object(2)
memory usage: 5.2+ KB


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562 entries, 0 to 561
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Path        562 non-null    object
 1   Message-ID  562 non-null    object
 2   Date        562 non-null    object
 3   From        562 non-null    object
 4   To          561 non-null    object
 5   Cc          19 non-null     object
 6   Bcc         19 non-null     object
 7   X-From      562 non-null    object
 8   X-To        562 non-null    object
 9   X-cc        562 non-null    object
 10  X-bcc       562 non-null    object
 11  Subject     562 non-null    object
 12  Message     562 non-null    object
dtypes: object(13)
memory usage: 57.2+ KB


In [13]:
sentence_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4038 entries, 0 to 4037
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Message-ID   4038 non-null   object
 1   Sentence-ID  4038 non-null   int64 
 2   Sentence     4038 non-null   object
dtypes: int64(1), object(2)
memory usage: 94.8+ KB


In [18]:
from db import connect
engine = connect()

In [19]:
df.to_sql('mail', con=engine, if_exists='append', index=False)

In [20]:
sentence_df.to_sql('sentence', con=engine, if_exists='append', index=False)

In [21]:
triple_df.to_sql('triple', con=engine, if_exists='append', index=False)

In [10]:
#output = pd.merge(df[['Path','Message-ID','Subject']],pd.merge(sentence_df,triple_df,on=['Message-ID','Sentence-ID']),on='Message-ID')
#output.to_csv('output/allen_p_inbox_triple4.csv')
#output.groupby(['Path','Sentence-ID','Sentence','Triple']).agg({'Triple':'unique'}).to_csv('output/allen_p_inbox_triple4.csv')